In [3]:
import astropy.table as tb
from astropy.time import Time


In [4]:
dssi_sessions = tb.Table.read("../2024-07-08 Process DSSI olist logs/DSSI sessions.csv")

tic_sessions = dssi_sessions[(dssi_sessions["TIC ID"].mask == False) & (dssi_sessions["MidTime JD"] > 0)]
tic_sessions["MidTime Byear"] = Time(tic_sessions["MidTime JD"], format="jd").byear
tic_sessions#[tic_sessions["Target Name"] == "TIC 269811101"]


Target Name,TIC ID,Speckle Session,StartTime JD,MidTime JD,EndTime JD,MidTime UTC,Num Sequences,MidTime Byear
str12,str13,int32,float64,float64,float64,str19,int32,float64
H300038,TIC 328181241,12,2459849.8666666667,2459849.8680555555,2459849.8694444443,2022-09-27 08:50:00,3,2022.739252707312
H300043,TIC 168789840,20,2459849.929861111,2459849.93125,2459849.9326388887,2022-09-27 10:21:00,3,2022.739425727967
H300045,TIC 454140642,22,2459849.9381944444,2459849.939236111,2459849.9402777776,2022-09-27 10:32:30,3,2022.7394475932147
H300046,TIC 139650665,29,2459849.961111111,2459849.963194445,2459849.965277778,2022-09-27 11:07:00,3,2022.7395131889587
H300047,TIC 178953404,30,2459849.967361111,2459849.9697916666,2459849.972222222,2022-09-27 11:16:30,5,2022.7395312515532
H300051,TIC 200094011,39,2459850.019444444,2459850.020833333,2459850.0222222223,2022-09-27 12:30:00,3,2022.7396709990046
H300036,TIC 389836747,43,2459850.834027778,2459850.834027778,2459850.834027778,2022-09-28 08:01:00,1,2022.7418974516095
H300032,TIC 283940788,46,2459850.8486111113,2459850.8506944445,2459850.8527777777,2022-09-28 08:25:00,5,2022.7419430834304
H300033,TIC 284814380,48,2459850.8583333334,2459850.8604166666,2459850.8625,2022-09-28 08:39:00,5,2022.7419697019925


In [5]:
ephemerides = tb.vstack([
    tb.Table.read("../../django/TargetDB/Data Files/Binary Parameters/Kostov 2022 Binary Parameters.csv"),
    tb.Table.read("../../django/TargetDB/Data Files/Binary Parameters/Kostov 2023 Binary Parameters.csv"),
])
ephemerides

Local ID,Member,Period,T0 Primary,T0 Secondary,Depth Primary,Depth Secondary,Duration Primary,Duration Secondary
str14,str1,float64,float64,float64,int32,int32,float64,float64
TIC 9493888,A,2.098992,1816.2345,1817.283786,146,117,2.9,2.7
TIC 25818450,A,10.132402,1769.9109,1776.391584,12,9,--,--
TIC 27543409,A,2.122862,1493.1001,1494.153889,50,15,--,--
TIC 31928452,A,2.8823,1337.9129,1339.358085,30,25,2.1,1.6
TIC 52856877,A,5.186818,1791.059,1793.652928,220,90,5.5,5.5
TIC 63459761,A,4.244072,1715.1118,1717.168477,15,10,7.0,9.2
TIC 75740921,A,0.93308,1519.6949,1520.16256,87,59,2.7,2.6
TIC 78568780,A,2.88838,1468.5374,1469.962816,57,20,3.9,3.7
TIC 79140936,A,3.54389,1468.3795,1469.78607,23,7,3.7,3.4


In [6]:
eclipses = tb.Table(names=["TIC ID", "Speckle Session", "Member", "Component", "Eclipse Beg JD", "Eclipse End JD", "Session Beg JD", "Session Mid JD", "Session End JD", "Session Mid UTC", "Session Mid Byear", "T0", "Period", "Half Duration Days", "Depth", "Cycles", "Eclipse Fraction", "FOM"],
                    dtype=["str", "int", "str", "str", "float", "float", "float", "float", "float", "str", "float", "float", "float", "float", "float", "float", "float", "float"],
)
for tic_session in tic_sessions:
    id = tic_session["TIC ID"]
    t = tic_session["MidTime JD"]
    for member in ["A", "B", "C"]:
        ephem = ephemerides[(ephemerides["Local ID"] == id) & (ephemerides["Member"] == member)]
        if len(ephem) == 0:
            continue
        ephem = ephem[0]
        period = ephem["Period"]
        for comp_letter, system in {"a": "Primary", "b": "Secondary"}.items():
            if ephem[f"T0 {system}"] > 0 and ephem[f"Duration {system}"] > 0:
                t0 = float(ephem[f"T0 {system}"]) + 2457000
                half_duration = ephem[f"Duration {system}"] / 24 / 2
                depth_name = f"Depth {system}"
                if depth_text := ephem[depth_name]:
                    depth = int(depth_text) / 1000 # kostov gives depth as ppt
                else:
                    depth = 0
                cycles = round((t - t0) / period)
                eclipse_beg = t0 + (period * cycles) - half_duration
                eclipse_end = eclipse_beg + 2 * half_duration
                confirmed = eclipse_beg < t < eclipse_end
                if confirmed:
                    mid_eclipse = (eclipse_beg + eclipse_end) / 2
                    eclipse_fraction = abs(t - mid_eclipse) / half_duration
                    fom = depth * (1 - eclipse_fraction) # this assumes linear slope of light curve, better would be gaussian
                    eclipses.add_row([id, tic_session["Speckle Session"], member, comp_letter, eclipse_beg, eclipse_end, tic_session["StartTime JD"], t, tic_session["EndTime JD"], tic_session["MidTime UTC"], tic_session["MidTime Byear"], t0, period, half_duration, depth, (t - t0) / period, eclipse_fraction, fom])
print(len(eclipses), len(tic_sessions))

eclipses.sort("FOM", reverse=True)
eclipses.write("tic eclipses.csv", overwrite=True)

eclipses

# foo = eclipses[["TIC ID", "Member", "Component", "Session Mid JD", "Period", "Half Duration Days", "Depth", "Eclipse Fraction", "FOM"]]
# foo#[foo["TIC ID"] == "TIC 286470992"]

34 161


TIC ID,Speckle Session,Member,Component,Eclipse Beg JD,Eclipse End JD,Session Beg JD,Session Mid JD,Session End JD,Session Mid UTC,Session Mid Byear,T0,Period,Half Duration Days,Depth,Cycles,Eclipse Fraction,FOM
str13,int32,str1,str1,float64,float64,float64,float64,float64,str19,float64,float64,float64,float64,float64,float64,float64,float64
TIC 269811101,341,A,b,2459898.5386933335,2459898.7220266666,2459898.617361111,2459898.625,2459898.632638889,2022-11-15 03:00:00,2022.8727447972383,2458768.165777,1.094351,0.09166666666666667,0.175,1032.995102119765,0.0584727254780856,0.164767273041335
TIC 125952257,288,A,a,2459895.859835,2459896.059835,2459895.975,2459895.9798611114,2459895.984722222,2022-11-12 11:31:00,2022.865502646968,2458492.877,2.161915,0.09999999999999999,0.14,649.0092631354693,0.20026111509650948,0.11196344388648868
TIC 278465736,2188,A,a,2460458.8596166666,2460459.026283333,2460458.938888889,2460458.941319444,2460458.94375,2024-05-28 10:35:30,2024.4068400395577,2458711.5155,0.61421,0.08333333333333333,0.088,2844.997345279309,0.019566670060157776,0.0862781330347061
TIC 443862276,550,A,a,2460012.549172,2460012.724172,2460012.6166666667,2460012.630208333,2460012.64375,2023-03-09 03:07:30,2023.1848807133879,2458494.1607,3.073838,0.08750000000000001,0.09,493.9978971998953,0.07387047899620873,0.08335165689034121
TIC 251757935,74,B,a,2459850.912211333,2459851.1080446662,2459850.976388889,2459850.980555556,2459850.984722222,2022-09-28 11:32:00,2022.7422986313702,2458792.6865,1.524962,0.09791666666666667,0.116,693.9806077501912,0.3020164497355197,0.08096609183067971
TIC 367448265,547,B,a,2460012.572036667,2460012.7512033335,2460012.604861111,2460012.604861111,2460012.604861111,2023-03-09 02:31:00,2023.1848113149938,2458816.8633,1.86552,0.08958333333333333,0.205,640.9695747625664,0.6335876033056614,0.0751145413223394
TIC 178953404,30,B,b,2459849.898607,2459850.061107,2459849.967361111,2459849.9697916666,2459849.972222222,2022-09-27 11:16:30,2022.7395312515532,2458477.727164,28.005157,0.08125,0.075,48.99964059000981,0.12388102137125455,0.06570892339715591
TIC 125952257,293,A,a,2459895.859835,2459896.059835,2459896.0090277777,2459896.013541667,2459896.0180555554,2022-11-12 12:19:30,2022.8655948612727,2458492.877,2.161915,0.09999999999999999,0.14,649.0248421732255,0.537066669203341,0.06481066631153226
TIC 470710327,1112,A,a,2460236.4877186664,2460236.729385333,2460236.6041666665,2460236.6083333334,2460236.6125,2023-10-19 02:36:00,2023.7981124969767,2458765.1668,1.104686,0.12083333333333333,0.06,1331.9998020556266,0.001809650967861044,0.059891420941928335


In [7]:
eclipses[eclipses["TIC ID"] == "TIC 367448265"]

TIC ID,Speckle Session,Member,Component,Eclipse Beg JD,Eclipse End JD,Session Beg JD,Session Mid JD,Session End JD,Session Mid UTC,Session Mid Byear,T0,Period,Half Duration Days,Depth,Cycles,Eclipse Fraction,FOM
str13,int32,str1,str1,float64,float64,float64,float64,float64,str19,float64,float64,float64,float64,float64,float64,float64,float64
TIC 367448265,547,B,a,2460012.572036667,2460012.7512033335,2460012.604861111,2460012.604861111,2460012.604861111,2023-03-09 02:31:00,2023.1848113149938,2458816.8633,1.86552,0.08958333333333333,0.205,640.9695747625664,0.6335876033056614,0.0751145413223394


In [8]:
table_1 = tb.Table.read("Eclipses from Table 1.csv")

table_1["JD"] = Time(table_1["Byear"], format="byear").jd

table_1


TIC ID,Mag,Byear,Chisq,Seeing,Theta,Rho,Delta m,Lambda,Delta lambda,Notes,Eclipse,JD
str13,float64,float64,float64,float64,float64,float64,str5,int32,int32,str70,str2,float64
TIC 25818450,11.45,2022.7447,6.19,0.93,181.0,0.0633,1.51,692,40,0.06\\,--,2459851.8576367144
TIC 25818450,11.45,2022.7447,0.67,1.22,181.2,0.0639,1.33,880,50,0.08\\,--,2459851.8576367144
TIC 45160946,13.7,2023.9106,--,--,163.1,0.1869,1.8,824,170,$\cdots$ \textcolor{red}{Noisy} \\,--,2460277.693516273
TIC 75740921,13.22,2023.9106,--,--,95.9,0.3986,0.4,824,170,$\cdots$ \textcolor{red}{Noisy} \\,--,2460277.693516273
TIC 78568780,11.51,2022.7479,3.79,0.96,246.9,0.2632,1.26,692,40,0.25 \textcolor{violet}{Aa Eclipse (4\%)}\\,Aa,2459853.0264117504
TIC 78568780,11.51,2022.7479,0.19,0.89,247.0,0.2631,1.2,880,50,0.23\\,--,2459853.0264117504
TIC 89278612,11.12,2022.7443,3.41,0.82,184.7,0.0894,0.61,692,40,0.07\\,--,2459851.7115398347
TIC 89278612,11.12,2022.7443,0.12,0.83,186.2,0.0894,0.55,880,50,0.07\\,--,2459851.7115398347
TIC 97356407,6.31,2022.8655,93.14,1.22,228.9,0.1103,4.33,692,40,0.13\\,--,2459895.978894327


In [9]:
# # go through each Table 1 measurement and calculate if it's in eclipse
# # this is different than the above work because it includes non-DSSI observations

# tess_offset = 2457000

# table_1.sort(["TIC ID", "Byear", "Lambda"])
# table_1["Eclipse System"] = "xx"
# table_1["Eclipse Depth"] = 0.0
# table_1["Eclipse Fraction"] = 0.0
# table_1["Eclipse FOM"] = 0.0

# tic_id, byear = "", 0.0
# for observation in table_1:
#     if (tic_id == observation["TIC ID"]) & (byear == observation["Byear"]):
#         continue # skip duplicate entries in table for different filters
#     tic_id = observation["TIC ID"]
#     byear = observation["Byear"]
#     for ephem in ephemerides[ephemerides["Local ID"] == observation["TIC ID"]]:
#         for system in ["Primary", "Secondary"]:
#             name = f"T0 {system}"
#             if t0 := ephem[name] > 0:
#                 half_duration = ephem[f"Duration {system}"] / 24 / 2
#                 depth_name = f"Depth {system}"
#                 if depth_text := ephem[depth_name]:
#                     depth = int(depth_text) / 1000 # kostov gives depth as ppt
#                 else:
#                     depth = 0
#                 cycles = round((t - t0) / period)
#                 eclipse_beg = t0 + (period * cycles) - half_duration
#                 eclipse_end = eclipse_beg + 2 * half_duration
#                 confirmed = eclipse_beg < t < eclipse_end
#                 if confirmed:
#                     mid_eclipse = (eclipse_beg + eclipse_end) / 2
#                     eclipse_fraction = abs(t - mid_eclipse) / half_duration
#                     fom = depth * (1 - eclipse_fraction) # this assumes linear slope of light curve, better would be gaussian
#                     eclipses.add_row([id, tic_session["Speckle Session"], member, comp_letter, eclipse_beg, eclipse_end, tic_session["StartTime JD"], t, tic_session["EndTime JD"], tic_session["MidTime UTC"], tic_session["MidTime Byear"], t0, period, half_duration, depth, (t - t0) / period, eclipse_fraction, fom])


#             t0 += tess_offset
#             period = ephems["Period"]
#             cycles = (session_mid - t0) / period



# table_1.sort("DSSI FOM", reverse=True)

# table_1[table_1["DSSI Depth"] > 0]

# # table_1[table_1["Lambda"] == 824]
# # table_1[table_1["Eclipse"].mask == False]

## Now, do a loose join to Table 1 to look for agreement

In [10]:
byear_tolerance = 0.01

eclipses["Byear"] = eclipses["Session Mid Byear"]
# tb.join(table_1, eclipses, keys=["TIC ID"], join_funcs={"TIC ID": lambda x, y: x==y}, join_type="outer")#, "Byear": lambda x, y: abs(x - y) < 0.0001})

table_1["DSSI System"] = "xx"
table_1["DSSI Depth"] = 0.0
table_1["DSSI Fraction"] = 0.0
table_1["DSSI FOM"] = 0.0

table_1.sort(["TIC ID", "Byear", "Lambda"])

tic_id, byear = "", 0.0
for observation in table_1:
    if (tic_id == observation["TIC ID"]) & (byear == observation["Byear"]):
        continue # skip duplicate entries in table for different filters
    tic_id = observation["TIC ID"]
    byear = observation["Byear"]
    # print(f"{tic_id} {row["Byear"]}")
    for eclipse in eclipses[eclipses["TIC ID"] == tic_id]:
        if abs(eclipse["Byear"] - observation["Byear"]) < byear_tolerance:
            # print(f"  {eclipse["Session Mid UTC"]}")
            observation["DSSI System"] = eclipse["Member"] + eclipse["Component"]
            observation["DSSI Depth"] = eclipse["Depth"]
            observation["DSSI Fraction"] = eclipse["Eclipse Fraction"]
            observation["DSSI FOM"] = eclipse["FOM"]
            break

table_1.sort("DSSI FOM", reverse=True)

table_1[table_1["DSSI Depth"] > 0]

# table_1[table_1["Lambda"] == 824]
# table_1[table_1["Eclipse"].mask == False]

TIC ID,Mag,Byear,Chisq,Seeing,Theta,Rho,Delta m,Lambda,Delta lambda,Notes,Eclipse,JD,DSSI System,DSSI Depth,DSSI Fraction,DSSI FOM
str13,float64,float64,float64,float64,float64,float64,str5,int32,int32,str70,str2,float64,str2,float64,float64,float64
TIC 367448265,7.91,2023.1848,6.58,0.77,10.5,0.6728,2.99,692,40,0.52 \textcolor{violet}{Ba Eclipse (24\%)}\\,Ba,2460012.6007283977,Ba,0.205,0.6335876033056614,0.0751145413223394
TIC 470710327,9.79,2023.8012,1.77,0.94,305.2,0.5082,1.08,692,40,0.48 \textcolor{violet}{Aa Eclipse (7\%)}\\,Aa,2460237.7360197264,Aa,0.06,0.001809650967861044,0.059891420941928335
TIC 470710327,9.79,2023.798,0.78,1.08,305.2,0.5103,1.15,692,40,0.55 \textcolor{violet}{Aa Eclipse (6\%)}\\,Aa,2460236.5672446904,Aa,0.06,0.001809650967861044,0.059891420941928335
TIC 470710327,9.79,2023.7983,0.93,1.55,305.2,0.5096,1.18*,692,40,{\bf0.79} \textcolor{violet}{Aa Eclipse (1\%)}\\,Aa,2460236.6768173496,Aa,0.06,0.001809650967861044,0.059891420941928335
TIC 470710327,9.79,2023.8015,62.43,0.78,305.2,0.5064,1.22,692,40,0.39 \textcolor{violet}{Aa Eclipse (1\%)}\\,Aa,2460237.845592386,Aa,0.06,0.001809650967861044,0.059891420941928335
TIC 470710327,9.79,2023.7986,0.95,1.14,305.2,0.5065,1.23,692,40,0.58\\,--,2460236.7863900093,Aa,0.06,0.001809650967861044,0.059891420941928335
TIC 470710327,9.79,2023.8013,7.03,0.73,305.2,0.5068,1.09,692,40,0.37 \textcolor{violet}{Aa Eclipse (6\%)}\\,Aa,2460237.7725439463,Aa,0.06,0.001809650967861044,0.059891420941928335
TIC 470710327,9.79,2023.8008,1.78,0.88,305.2,0.5079,1.26,692,40,0.45 \textcolor{violet}{Aa Eclipse ($<1$\%)}\\,Aa,2460237.5899228468,Aa,0.06,0.001809650967861044,0.059891420941928335
TIC 317863971,10.32,2023.1823,0.04,0.71,184.5,2.5793,3.92*,692,40,\bf1.83\\,--,2460011.6876229006,Ba,0.06,0.21098245523477857,0.047341052685913286


In [11]:
ephemerides[ephemerides["Local ID"] == "TIC 168789840"]

Local ID,Member,Period,T0 Primary,T0 Secondary,Depth Primary,Depth Secondary,Duration Primary,Duration Secondary
str14,str1,float64,float64,float64,int32,int32,float64,float64
TIC 168789840,A,1.305883,2151.193,--,10,--,--,--
TIC 168789840,B,1.570013,2151.868,2152.653321,40,20,3.2,3.0
TIC 168789840,C,8.217111,2151.446,2155.54716,55,10,3.4,3.3


In [12]:
#check out the 24% eclipse claim for TIC 367448265

my_tic = "TIC 367448265"

rows = ephemerides[ephemerides["Local ID"] == my_tic]
rows.pprint_all()
print()

eclipses[eclipses["TIC ID"] == my_tic].pprint_all()
print()

session_mid = eclipses[eclipses["TIC ID"] == my_tic]["Session Mid JD"][0]
print(f"Observation = {session_mid}")
print()

tess_offset = 2457000

for observation in rows:
    for system in ["Primary", "Secondary"]:
        name = f"T0 {system}"
        if observation[name] > 0:
            t0 = observation[name] + tess_offset
            period = observation["Period"]
            cycles = (session_mid - t0) / period
            print(f"{observation["Member"]} {system:<11s} {t0:.6f} {period:.6f} {cycles:8.3f} {observation['Duration ' + system] / 24}")

   Local ID   Member  Period  T0 Primary T0 Secondary Depth Primary Depth Secondary Duration Primary Duration Secondary
------------- ------ -------- ---------- ------------ ------------- --------------- ---------------- ------------------
TIC 367448265      A 0.418238   1816.293           --            20              --               --                 --
TIC 367448265      B  1.86552  1816.8633  1817.804268           205             180              4.3                4.3

    TIC ID    Speckle Session Member Component   Eclipse Beg JD    Eclipse End JD     Session Beg JD    Session Mid JD    Session End JD    Session Mid UTC   Session Mid Byear       T0       Period  Half Duration Days Depth       Cycles       Eclipse Fraction         FOM               Byear       
------------- --------------- ------ --------- ----------------- ------------------ ----------------- ----------------- ----------------- ------------------- ------------------ ------------ ------- ------------------- --

In [47]:
from astropy.table.row import Row
from astropy.time import Time
import astropy.units as u

ephem = tb.QTable.read("../../django/TargetDB/Data Files/Binary Parameters/Kostov ephemerides.csv")
ephem["Period"] = ephem["Period"] * u.day
ephem["Duration"] = ephem["Duration"] * u.hour

def calc_eclipse_timings(
    t0: Time = None,
    period: u.Quantity = None,
    duration: u.Quantity = None,
    beg_time: Time = None,
    end_time: Time = None,
    count: int = 0,
    row = None,
) -> list[tuple[Time, Time]]:
    """Given an initial time (t0), a period and optionally a duration, return a list of eclipses.
    The results can be a fixed number of eclipses, or all eclipses falling between a beginning and end time.
    Period & duration are Astropy Quantity, t0, beg_time & end_time are Astropy Time objects, and count is an int.
    """
    # process & validate inputs
    if isinstance(row, Row):
        t0 = row["T0"]
        if not isinstance(t0, Time):
            t0 = Time(t0, format="jd", scale="utc")
        period = row["Period"]
        if "Duration" in row.columns:
            duration = row["Duration"]
    if beg_time is None:
        raise ValueError("Must specify begin time")
    if count > 0:
        if end_time is not None:
            raise ValueError("Cannot specify both a count and an end time")
        use_count = True
    else:
        if count < 0:
            raise ValueError("Count cannot be negative")
        if end_time is None:
            raise ValueError("If count is zero, end time must be specified")
        use_count = False


    if duration > 0:
        half_duration = duration / 2
    else:
        half_duration = 2.5 * u.hour
    t = t0 + (round((beg_time.jd - t0.jd) / period.value) * period)
    # while t - half_duration < beg_time:
    #     t += period
    eclipses = []
    while True:
        if duration is not None:
            eclipse_beg = t - half_duration
            eclipse_end = t + half_duration
        else:
            eclipse_beg = t
            eclipse_end = t
        eclipses.append((eclipse_beg, eclipse_end))
        t += period
        if use_count:
            if len(eclipses) >= count:
                break
        else:
            if end_time < t + half_duration:
                break
    return eclipses

ephem
for hrcam in table_1[table_1["Lambda"]==824]:
    tic_id = hrcam["TIC ID"]
    time = Time(hrcam["Byear"], format="byear")
    found = False
    for eclipse in ephem[ephem["Local ID"] == tic_id]:
        for beg, end in calc_eclipse_timings(row=eclipse, beg_time=time, count=1):
            if beg <= time <= end:
                found = True
                print(f"{beg.iso} {time.iso} {end.iso}")
                print(f"{tic_id:<14s} on {time.iso} (Byear={time.byear:9.4f}) {eclipse["System"]}{eclipse["Member"]} depth={eclipse["Depth"]}")
                break
        if found:
            break
    if not found:
        print(f"{tic_id:<14s} on {time.iso} (Byear={time.byear:9.4f}) not in eclipse")


2023-06-27 14:27:29.722 2023-06-27 19:20:07.196 2023-06-27 19:27:29.722
TIC 257776944  on 2023-06-27 19:20:07.196 (Byear=2023.4879) Ba depth=0.038
TIC 255532033  on 2023-08-30 19:58:56.320 (Byear=2023.6632) not in eclipse
TIC 300446218  on 2023-11-29 03:46:04.113 (Byear=2023.9105) not in eclipse
TIC 201310151  on 2023-07-28 02:53:52.052 (Byear=2023.5709) not in eclipse
2023-11-24 02:08:10.810 2023-11-24 02:47:58.535 2023-11-24 06:56:10.810
TIC 200094011  on 2023-11-24 02:47:58.535 (Byear=2023.8967) Ab depth=0.055
TIC 97356407   on 2023-11-29 02:53:28.421 (Byear=2023.9104) not in eclipse
TIC 45160946   on 2023-11-29 04:38:39.806 (Byear=2023.9106) not in eclipse
2023-04-28 19:38:39.840 2023-04-28 19:59:19.951 2023-04-28 23:20:39.840
TIC 461614217  on 2023-04-28 19:59:19.951 (Byear=2023.3237) Aa depth=0.02
TIC 75740921   on 2023-11-29 04:38:39.806 (Byear=2023.9106) not in eclipse
2023-04-28 19:38:27.312 2023-04-28 19:59:19.951 2023-04-28 23:38:27.312
TIC 97356407   on 2023-04-28 19:59:19.

In [54]:
foo = Time(2458496.1994, format="jd")
bar = 2.077867
baz = Time("2023-11-24 05:25:45.613", format="iso", scale="utc")

cycles = round((baz.jd - foo.jd) / bar)

Time(foo.jd + bar * 855, format="jd", scale="utc").iso, baz.iso



('2023-11-24 06:36:59.184', '2023-11-24 05:25:45.613')